In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('weights/best.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(["img_sample/extract840.jpg", 'img_sample/extract841.jpg'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

# Inference Arguments

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on 'bus.jpg' with arguments
result = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.4)

# Boxes

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.boxes)  # print the Boxes object containing the detection bounding boxes

# Masks

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-seg Segment model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.masks)  # print the Masks object containing the detected instance masks

# Keypoints

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-pose Pose model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.keypoints)  # print the Keypoints object containing the detected keypoints


# Probs

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-cls Classify model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.probs)  # print the Probs object containing the detected class probabilities


# Plotting Results

In [ ]:
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    # im.save('results.jpg')  # save image

# Streaming Source for-loop

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ouster import client
from ouster import pcap
# from ouster.sdk import viz
from ouster.client import Scans
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing



# Load the YOLOv8 model
model = YOLO('weights/best_x_0_92mAP50.pt')

# Paths to pcap and json files
# test_2_wilanow
metadata_path = "C:/Users/szyme/Ouster/data/PKR_test2/test_2_wilanow.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test2/test_2_wilanow.pcap"

# Making PacketSource from data
with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())


fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

with closing(client.Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/YOLOv8/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for scan in scans:
        # Create signal image
        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = client.destagger(pcap_file.metadata, sig_field)

        # Define a scaling factor based on the values (adjust this as needed)
        scaling_factor = 0.004
        # Lower the elements based on their values
        lowered_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered) # BASIC
        # Convert to uint8
        signal_image = lowered_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))
        # print(signal_image.shape)

        # Create range image (for localization, distance measurement)
        range_field = scan.field(client.ChanField.RANGE)
        range_image = client.destagger(pcap_file.metadata, range_field)


        # Predict image
        results = model.predict(combined_img, save=False, imgsz=1024, conf=0.5)
        # results = model.track(source=combined_img, persist=True, iou=0.5, conf=0.4, tracker='botsort.yaml')
            
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Stream results
        if True:
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    vid_writer.release()
    cv2.destroyAllWindows()